## RL train the d20 model

I'm going to approach this like `challenge-30-sft-train-d20/sft-train-d20.ipynb`. I'll use the 8xH100 in lambda cloud and run all the scripts from this notebook.

Follow the instructions here to get the machine ready: `challenge-28-midtrain-d20/midtrain-d20.ipynb`

In [ ]:
import os
os.environ["PYTHONPATH"] = "../my_nanochat"

### RL train with tiny number of steps

Just to confirm the process runs through to completion, logs to wandb, etc.

In [ ]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_rl -- \
--source=sft --model-tag=d20 --num-steps=10

### RL train for real

In [ ]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_rl -- \
--source=sft --model-tag=d20

### Do chat eval on RL model with GSM8K task

My expectation (or at least hope) is to see accuracy go up from the centered accuracy of .05 (see `challenge-34-understand-reporting/run-evals-on-d20.ipynb`)

In [ ]:
!torchrun --standalone --nproc_per_node=8 -m scripts.my_chat_eval -- \
--source=rl --model-tag=d20 --task-name=GSM8K

### Rebuild final report

Total time will be completely wrong

In [ ]:
!python -m my_nanochat.my_report

In [ ]:
from IPython.display import Markdown
Markdown(open('report.md').read())